## 测试mps态

In [1]:
import Library.TensorNetwork as TN
import torch as tc

In [2]:
mps = TN.rand_mps_pack(number=2, length=3, chi=4, phydim=2)
print(mps.get_norm())
print(mps.center)
print(TN.inner_mps_pack(mps, mps))

tensor([1.0000, 1.0000])
1
tensor([1.0000+0.j, 1.0000+0.j])


将大张量表示的量子态转化为mps态，并计算误差

In [3]:
number = 2
length = 3
dtype = tc.complex64
device = tc.device('cpu')
shape = [number, 2 ** length]
states = tc.randn(shape, dtype=dtype, device=device)
shape_ = [number] + [2]*length
norm = tc.sum(states * states.conj(), dim=1, keepdim=True)
states = states / tc.sqrt(norm)
states = states.reshape(shape_)
print(states.shape)
mps_ = TN.TensorTrain_pack(tensor_packs=[states], length=length, phydim=2, center=-1, chi=8, device=device, dtype=dtype, initialize=True)
mps_copy = TN.copy_from_mps_pack(mps_)
mps_copy.merge_all()
print(mps_copy.node_list[0].shape)
print(tc.dist(mps_copy.node_list[0].squeeze(), states))

torch.Size([2, 2, 2, 2])
torch.Size([2, 1, 2, 2, 2, 1])
tensor(3.9952e-07)


将n体门转化为mpo，并计算和转化前的误差

In [4]:
n = 2
shape = [2] * (n*2)
n_body_op = tc.rand(shape, dtype=dtype, device=device)
print(list(n_body_op.shape))
gate_list = TN.n_body_gate_to_mpo(n_body_op, n, dtype=dtype, device=device)
print('')
for gate in gate_list:
    print(gate.shape)
gate_U = tc.einsum('abcd, defg -> abecfg', gate_list[0], gate_list[1])
print(gate_U.squeeze().shape)
print(tc.dist(gate_U.squeeze(), n_body_op))

[2, 2, 2, 2]

torch.Size([1, 2, 2, 4])
torch.Size([4, 2, 2, 1])
torch.Size([2, 2, 2, 2])
tensor(0.)


In [6]:
number = 1
length = 2
dtype = tc.complex64
device = tc.device('cpu')
shape = [number, 2 ** length]
states = tc.randn(shape, dtype=dtype, device=device)
shape_ = [number] + [2]*length
norm = tc.sum(states * states.conj(), dim=1, keepdim=True)
states = states / tc.sqrt(norm)
print(tc.einsum('ni, ni -> n', states, states.conj()))

states = states.reshape(shape_)
print(states.shape)

mps = TN.TensorTrain_pack(tensor_packs=[states], length=length, phydim=2, center=-1, chi=8, device=device, dtype=dtype, initialize=True)

copy = TN.copy_from_mps_pack(mps)
copy.merge_all()
print(tc.dist(copy.node_list[0].squeeze(), states), '\n')

n = 2
shape = [2] * (n*2)
A = tc.rand(shape, dtype=dtype, device=device).reshape([2 ** n, 2 ** n])
H = 0.5 * (A + A.conj().T)

print(list(n_body_op.shape))

gate_U = tc.matrix_exp(1.j * H).reshape(shape)
print(gate_U.shape)
gates = TN.n_body_gate_to_mpo(gate_U, 2, dtype=dtype, device=device)
print('gates:')
for gate in gates:
    print(gate.squeeze().shape)
# print(tc.einsum('ij, kj-> ik', gate_U, gate_U.conj()))
states_act_U = tc.einsum('aci, ibd, ncd -> nab', gates[0].squeeze(), gates[1].squeeze(), states)
states_act_U = states_act_U.reshape([number] + [2]*length)
print('states_act_U.shape')
print(states_act_U.shape)
print()

mps.act_n_body_gate(n_body_op, [0, 1])
print('mps_act_U node shape:')
for node in mps.node_list:
    print(node.shape)
print()

mps.merge_nodes((0,1))
print('merge(0,1)->node0.shape')
print(mps.node_list[0].squeeze().shape)
print()

print('distance between mps and states_act_U')
print(tc.dist(mps.node_list[0].squeeze(), states_act_U.squeeze()))

tensor([1.+0.j])
torch.Size([1, 2, 2])
tensor(1.7568e-07) 

[2, 2, 2, 2]
torch.Size([2, 2, 2, 2])
gates:
torch.Size([2, 2, 4])
torch.Size([4, 2, 2])
states_act_U.shape
torch.Size([1, 2, 2])

mps_act_U node shape:
torch.Size([1, 1, 2, 8])
torch.Size([1, 8, 2, 1])

merge(0,1)->node0.shape
torch.Size([2, 2])

distance between mps and states_act_U
tensor(1.7061)


In [7]:
a = tc.rand([1,2,3,4,5])
b = a.permute(0, 1, 3, 4, 2)
print(b.shape)

torch.Size([1, 2, 4, 5, 3])


In [8]:
def convert_cycle_to_full_sequence(cycle, length):
    # 初始化完整序列为原始顺序
    full_sequence = list(range(length))
    
    # 应用轮换表示
    for i in range(len(cycle) - 1):
        full_sequence[cycle[i]], full_sequence[cycle[i + 1]] = full_sequence[cycle[i + 1]], full_sequence[cycle[i]]
    
    return full_sequence

print(convert_cycle_to_full_sequence(cycle=[2, 3, 4], length=5))

[0, 1, 3, 4, 2]


merge和permute操作没有bug

In [10]:
a_list = [tc.rand(3, 1, 2, 2), tc.rand(3, 2, 2, 1)]
a = TN.TensorTrain_pack(tensor_packs=a_list, length=2, chi=4, device=tc.device('cpu'), dtype=tc.complex64, initialize=False)
a_copy = TN.copy_from_mps_pack(a)

print('before act gate:', 'site 0:', a.node_list[0].shape, 'site 1:', a.node_list[1].shape)
# a.connect([0, 3], [1, 1])
# a.connect([1, 3], [2, 1])
U = tc.rand(2, 2, 2, 2, device=tc.device('cpu'), dtype=tc.complex64)
pos = [0, 1]

a_copy.act_n_body_gate(U, pos)
print(a_copy.connect_graph)
a_copy.merge_all()
a__ = a_copy.node_list[0]
print(a__.shape)

gates = TN.n_body_gate_to_mpo(U, 2, dtype=tc.complex64, device=tc.device('cpu'))
U1 = gates[0]
print(U1.shape)
# U1 = tc.rand(2, 2, 4, device=tc.device('cpu'), dtype=tc.complex64)
a.add_node(gates[0].squeeze(), site=pos[0]+1, device=a.device, dtype=a.dtype)
# a.add_node(U1.squeeze(), site=1, device=tc.device('cpu'), dtype=tc.complex64)
a.connect([pos[0], 2], [pos[0]+1, 1])
a.merge_nodes((pos[0], pos[0]+1), is_gate=(False, True))
a.permute_legs(pos[0], cycle=[2, 3, 4])
# a.connect([0, 2], [1, 1])
# a.merge_nodes((0, 1), is_gate=(False, True))
# a.permute_legs(node_idx=0, cycle=[2, 3, 4])
U2 = gates[1]
print(U2.shape)
# U2 = tc.rand(4, 2, 2, device=tc.device('cpu'), dtype=tc.complex64)

a.add_node(gates[-1].squeeze(), site=pos[-1]+1, device=a.device, dtype=a.dtype)
a.connect([pos[-1], -2], [pos[-1]+1, 2])
a.connect([pos[-1]-1, -2], [pos[-1]+1, 0])
a.merge_nodes((pos[-1], pos[-1]+1), is_gate=(False, True))
a.permute_legs(pos[-1], cycle=[2, 3, 4])

# a.add_node(U2.squeeze(), site=2, device=tc.device('cpu'), dtype=tc.complex64)
# a.connect([1, -2], [2, 2])
# a.connect([0, -2], [2, 0])
# a.merge_nodes((1, 2), is_gate=(False, True))
# a.permute_legs(node_idx=1, cycle=[2, 3, 4])

b1 = a.node_list[0]
b2 = a.node_list[1]
print('gate shape:', 'gate 0:', U1.shape, 'gate 1:', U2.shape)
print('after act gate:', 'site 0:', b1.shape, 'site 1:', b2.shape)
print(a.connect_graph)
a.merge_nodes(node_pair=(0,1), is_gate=(False, False))
c = a.node_list[0]
d = tc.einsum('aijkl, alkmn -> aijmn', b1, b2)
print(c.shape, d.shape)
print(tc.dist(c, d))

print(tc.dist(a__.squeeze(), c.squeeze()))

before act gate: site 0: torch.Size([3, 1, 2, 2]) site 1: torch.Size([3, 2, 2, 1])
{(0, 1): [(3, 1)]}
torch.Size([3, 1, 2, 2, 1])
torch.Size([1, 2, 2, 4])
torch.Size([4, 2, 2, 1])
gate shape: gate 0: torch.Size([1, 2, 2, 4]) gate 1: torch.Size([4, 2, 2, 1])
after act gate: site 0: torch.Size([3, 1, 2, 4, 2]) site 1: torch.Size([3, 2, 4, 2, 1])
{(0, 1): [(4, 1), (3, 2)]}
torch.Size([3, 1, 2, 2, 1]) torch.Size([3, 1, 2, 2, 1])
tensor(2.9200e-07)
tensor(0.)


In [12]:
import time
length = 10
number = 5
shape = [number] + [2] * length
t1 = time.time()
a_list = [tc.rand(shape, device=tc.device('cpu'), dtype=tc.complex64)]
a = TN.TensorTrain_pack(tensor_packs=a_list, length=length, chi=16, device=tc.device('cpu'), dtype=tc.complex64, initialize=True)
gate = tc.eye(4, device=tc.device('cpu'), dtype=tc.complex64).reshape([2, 2, 2, 2])
for _ in range(1):
    for i in range(9):
        a.act_n_body_gate(gate=gate, pos=[i, i+1])
t2 = time.time()
print(t2-t1)
b = a_list[0]
for _ in range(9):
    b = tc.einsum('ab, nbc -> nac', gate.reshape([4, 4]), b.reshape([number, 4, -1])).reshape(shape)
t3 = time.time()
print(t3-t2)

0.048668622970581055
0.0
